In [1]:
import numpy as np
import scipy.linalg as la
import scipy.signal as signal
from numpy.linalg import inv
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft

### Spectrogram

In [ ]:
sample_rate = 1e6

# Generate tone plus noise
t = np.arange(1024*1000)/sample_rate # time vector
f = 50e3 # freq of tone
x = np.sin(2*np.pi*f*t) + 0.2*np.random.randn(len(t))

fft_size = 1024
num_rows = int(np.floor(len(x)/fft_size))
spectrogram = np.zeros((num_rows, fft_size))
for i in range(num_rows):
    spectrogram[i,:] = np.log10(np.abs(np.fft.fftshift(np.fft.fft(x[i*fft_size:(i+1)*fft_size])))**2)
spectrogram = spectrogram[:,fft_size//2:] # get rid of negative freqs because we simulated a real signal


plt.imshow(spectrogram, aspect='auto', extent = [0, sample_rate/2/1e6, 0, len(x)/sample_rate])
plt.xlabel("Frequency [MHz]")
plt.ylabel("Time [s]")
plt.show()
#plt.savefig('../_images/spectrogram.svg', bbox_inches='tight')

## Generate QPSK Symbols

In [ ]:
### No noise
num_symbols = 1000

x_int = np.random.randint(0, 4, num_symbols) # 0 to 3
print(x_int)
x_degrees = x_int*360/4.0 + 45 # 45, 135, 225, 315 degrees
x_radians = x_degrees*np.pi/180.0 # sin() and cos() takes in radians
x_symbols = np.cos(x_radians) + 1j*np.sin(x_radians) # this produces our QPSK complex symbols

fig, (ax1) = plt.subplots(1, 1, figsize=(4, 4))
fig.subplots_adjust(hspace=0.4)
ax1.plot(np.real(x_symbols), np.imag(x_symbols), '.')
ax1.set_ylabel("Q")
ax1.set_xlabel("I")
ax1.set_ylim(bottom=-1, top=1)
ax1.set_xlim(left=-1, right=1)
ax1.grid()
plt.show()

In [ ]:
### Add AWGN
n = (np.random.randn(num_symbols) + 1j*np.random.randn(num_symbols))/np.sqrt(2) # AWGN with unity power
noise_power = 0.01
r = x_symbols + n * np.sqrt(noise_power)

fig, (ax1) = plt.subplots(1, 1, figsize=(4, 4))
fig.subplots_adjust(hspace=0.4)
ax1.plot(np.real(r), np.imag(r), '.')
ax1.set_ylabel("Q")
ax1.set_xlabel("I")
ax1.set_ylim(bottom=-1, top=1)
ax1.set_xlim(left=-1, right=1)
ax1.grid()
plt.show()

In [ ]:
### Phase noise
phase_noise = np.random.randn(len(x_symbols)) * 0.15 # adjust multiplier for "strength" of phase noise
r = x_symbols * np.exp(1j*phase_noise)

fig, (ax1) = plt.subplots(1, 1, figsize=(4, 4))
fig.subplots_adjust(hspace=0.4)
ax1.plot(np.real(r), np.imag(r), '.')
ax1.set_ylabel("Q")
ax1.set_xlabel("I")
ax1.set_ylim(bottom=-1, top=1)
ax1.set_xlim(left=-1, right=1)
ax1.grid()
plt.show()

In [ ]:
### Phase noise plus AWGN
phase_noise = np.random.randn(len(x_symbols)) * 0.2 # adjust multiplier for "strength" of phase noise
r = x_symbols * np.exp(1j*phase_noise) + n * np.sqrt(noise_power)

fig, (ax1) = plt.subplots(1, 1, figsize=(4, 4))
fig.subplots_adjust(hspace=0.4)
ax1.plot(np.real(r), np.imag(r), '.')
ax1.set_ylabel("Q")
ax1.set_xlabel("I")
ax1.set_ylim(bottom=-1.2, top=1.2)
ax1.set_xlim(left=-1.2, right=1.2)
ax1.grid()
plt.show()

In [ ]:
import numpy as np
import pylab as pl
import scipy.signal.signaltools as sigtool
import scipy.signal as signal
from numpy.random import sample

#the following variables setup the system
Fc = 1000       #simulate a carrier frequency of 1kHz
Fbit = 50       #simulated bitrate of data
Fdev = 500      #frequency deviation, make higher than bitrate
N = 8          #how many bits to send
A = 1           #transmitted signal amplitude
Fs = 1000      #sampling frequency for the simulator, must be higher than twice the carrier frequency
A_n = 0.1       #noise peak amplitude
N_prntbits = 8 #number of bits to print in plots

def plot_data(y):
    #view the data in time and frequency domain
    #calculate the frequency domain for viewing purposes
    N_FFT = float(len(y))
    f = np.arange(0,Fs/2,Fs/N_FFT)
    w = np.hanning(len(y))
    y_f = np.fft.fft(np.multiply(y,w))
    y_f = 10*np.log10(np.abs(y_f[0:N_FFT/2]/N_FFT))
    pl.subplot(3,1,1)
    pl.plot(t[0:Fs*N_prntbits/Fbit],m[0:Fs*N_prntbits/Fbit])
    pl.xlabel('Time (s)')
    pl.ylabel('Frequency (Hz)')
    pl.title('Original VCO output versus time')
    pl.grid(True)
    pl.subplot(3,1,2)
    pl.plot(t[0:Fs*N_prntbits/Fbit],y[0:Fs*N_prntbits/Fbit])
    pl.xlabel('Time (s)')
    pl.ylabel('Amplitude (V)')
    pl.title('Amplitude of carrier versus time')
    pl.grid(True)
    pl.subplot(3,1,3)
    pl.plot(f[0:(Fc+Fdev*2)*N_FFT/Fs],y_f[0:(Fc+Fdev*2)*N_FFT/Fs])
    pl.xlabel('Frequency (Hz)')
    pl.ylabel('Amplitude (dB)')
    pl.title('Spectrum')
    pl.grid(True)
    pl.tight_layout()
    pl.show()
    
data_in = "01010101"

In [ ]:
len(np.arange(-Fs/Fbit,Fs/Fbit,1/Fs))

In [ ]:
#extend the data_in to account for the bitrate and convert 0/1 to frequency
m = []
for bit in data_in:
    if bit == "0":
        m=np.hstack((m,np.ones(int(Fs/Fbit))*(Fc+Fdev)))
    else:
        m=np.hstack((m,np.ones(int(Fs/Fbit))*(Fc-Fdev)))
#calculate the output of the VCO
t = np.arange((-2*N/Fbit),(2*N/Fbit),1/Fs)
print(len(m),len(t))
y=A * np.cos(2*np.pi*m*t)
plot_data(y)

In [ ]:
"""
Noisy Channel
"""
#create some noise
noise = (np.random.randn(len(y))+1)*A_n
snr = 10*np.log10(np.mean(np.square(y)) / np.mean(np.square(noise)))
print "SNR = %fdB" % snr
y=np.add(y,noise)
#view the data after adding noise
plot_data(y)